In [154]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,LeakyReLU,BatchNormalization,GlobalAveragePooling2D,GlobalMaxPooling2D,AveragePooling2D
from keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import DenseNet121

In [155]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,              
    restore_best_weights=True
)

In [156]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=64
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=64
)

Found 6433 files belonging to 5 classes.
Using 5469 files for training.
Found 6433 files belonging to 5 classes.
Using 964 files for validation.


In [157]:
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),         
    layers.RandomRotation(0.25),             
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.1)
])

In [158]:
normalization_layer = layers.Rescaling(1./255)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [159]:
weights_Add="/kaggle/input/densenet121-wts/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = DenseNet121(weights=weights_Add, include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [160]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(5, activation='softmax')  
])

In [161]:
model.summary()

Model: "sequential_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 64)             │     3,211,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_14 (LeakyReLU)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,249,157 (39.10 MB)

 Trainable params: 3,211,653 (12.25 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [162]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_ds,epochs=30,validation_data=val_ds,callbacks=[early_stop])

Epoch 1/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 82s 643ms/step - accuracy: 0.6587 - loss: 4.6767 - val_accuracy: 0.8963 - val_loss: 0.3977
Epoch 2/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 33s 369ms/step - accuracy: 0.8710 - loss: 0.4325 - val_accuracy: 0.9108 - val_loss: 0.2773
Epoch 3/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 33s 377ms/step - accuracy: 0.9065 - loss: 0.2936 - val_accuracy: 0.9180 - val_loss: 0.3057
Epoch 4/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 35s 399ms/step - accuracy: 0.9093 - loss: 0.2669 - val_accuracy: 0.9222 - val_loss: 0.2743
Epoch 5/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 33s 375ms/step - accuracy: 0.9099 - loss: 0.2967 - val_accuracy: 0.9077 - val_loss: 0.3006
Epoch 6/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 33s 369ms/step - accuracy: 0.9162 - loss: 0.2727 - val_accuracy: 0.9056 - val_loss: 0.3761
Epoch 7/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 33s 370ms/step - accuracy: 0.9186 - loss: 0.2973 - val_accuracy: 0.9305 - val_loss: 0.2406
Epoch 8/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 33s 372ms/step - accuracy: 0.9320 - loss: 0.2145 - val_accu